# Out-of-disrtibution test

This jupyter notebook provide out-of-distribution test. Namely, we train unsupervised models on ms1m dataset, considered as out-of-distribution detector, calculate features for validation datasets and for outliers dataset. Then we calculate distance between outliers and validation datasets features distribution. 

In [1]:
import json
import numpy as np
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
import pandas as pd

### Features extraction
Choose methods and datasets. As out-of-distribution dataset we use cplfw_anime_outliers. Also we discard theese outliers from cplfw dataset.


In [2]:
with open("config.json", "r") as read_file:
    config_test = json.load(read_file)['test_params']

methods = ['OneClassSVM', 'PCA', 'norms']
datasets = ['calfw', 'flickr', 'ms1m', 'cplfw', 'megaface']

In [3]:
! python3 create_models.py --methods {' '.join(methods)}
! python3 calculate_features.py --shuffle --methods {' '.join(methods)} --datasets {' '.join(datasets)} 


outliers: 100%|##################################| 6/6 [39:08<00:00, 391.35s/it]


### Experiments

As a distance between in-distribution and out-distribution we use ROC-AUC score. Namely, for a given binary classifier, 

In [6]:
def get_rocauc(distances_id, distances_ood):
    """Return roc score with 2 arrays as in-distribution
       and out-of-distribution arrays respectively"""
    y = np.array([1]*len(distances_id) + [0]*len(distances_ood))
    return roc_auc_score(1-y, np.concatenate((distances_id, distances_ood), axis=0))


def roc_auc_experiments(in_distr_features, ood_features, in_distr_fraction=config_test['proportion_of_outliers'],
                       n_experiments=config_test['n_experiments'], shuffle=False, extra_params=None):
    
    result = []
    emb_length = int(len(ood_features) / in_distr_fraction)
    
    for i in range(n_experiments):
        if shuffle:
            test_emb = in_distr_features[np.random.choice(len(in_distr_features), size=emb_length)]
        else:
            assert emb_length * n_experiments == len(in_distr_features)
            test_emb = in_distr_features[i*emb_length:(i+1)*emb_length]
            
        result.append(get_rocauc(test_emb, ood_features))
        
    return np.mean(result), np.std(result)


In [9]:
results = {}

for method in methods:
    ood_features = np.load('features/outliers/{}_dist.npy'.format(method))
    results[method] = {}
    for dataset in datasets:
        in_distr_features = np.load('features/{}/{}_dist.npy'.format(dataset, method))
        mean, std = roc_auc_experiments(in_distr_features, ood_features)
        conf_intv = 1.96 * std / np.sqrt(config_test['n_experiments'])
        
        results[method][dataset] = (np.round(mean, 3), np.round(conf_intv, 3))

results_df = pd.DataFrame.from_dict(results)


### results table


In [10]:
results_df

,OneClassSVM,PCA,norms
calfw,"(0.788, 0.0)","(0.878, 0.0)","(0.703, 0.0)"
flickr,"(0.707, 0.001)","(0.898, 0.0)","(0.72, 0.0)"
ms1m,"(0.776, 0.0)","(0.883, 0.0)","(0.751, 0.0)"
cplfw,"(0.774, 0.0)","(0.819, 0.0)","(0.747, 0.0)"
megaface,"(0.698, 0.001)","(0.808, 0.0)","(0.742, 0.0)"
